In [30]:
"""
Sprinter time between two user-defined lines (start/end) using hip tracking.
- Click 4 points on the first displayed frame:
    1) Start line endpoint A
    2) Start line endpoint B
    3) End line endpoint A
    4) End line endpoint B
  The corridor (ROI) is the quad [startA, startB, endB, endA].
- The script tracks the runner's hip using MediaPipe Pose.
- It records the video times when the hip crosses the start and end segments (within the endpoints),
  then prints and overlays the time difference.
- Only hip positions inside the clicked corridor are considered (to ignore background people).
- Optionally shows a bird's‑eye view of the corridor (homography warp) for visual assurance.

Controls:
  q   : quit
  r   : reset start/end detections
  w   : toggle bird's-eye warped preview window
  v   : toggle writing an annotated video (if enabled below)
"""
import cv2
import numpy as np
import math
import time
from collections import deque

# -------------- USER SETTINGS --------------
VIDEO_PATH = r"C:\Users\yenul\Downloads\edit one - Made with Clipchamp.mp4"  # Change to your path if needed
WRITE_ANNOTATED_VIDEO = False            # Set True to save an annotated MP4
OUT_VIDEO = "time_between_lines_annotated.mp4"

# MediaPipe Pose settings
POSE_MIN_DET = 0.5
POSE_MIN_TRK = 0.5
MODEL_COMPLEXITY = 1

# Smoothing for hip (exponential moving average)
EMA_ALPHA = 0.25

# Crossing detection threshold (pixels). Signed distance must exceed this before counting a flip
CROSSING_THRESH_PX = 2.0

# Projection tolerance to accept a crossing as "within the segment" (t in [-tol, 1+tol])
SEGMENT_T_TOL = 0.05
# -------------------------------------------

# ---- Helper math ----
def dist(a, b):
    ax, ay = a
    bx, by = b
    return math.hypot(bx - ax, by - ay)

def unit(v):
    vx, vy = v
    n = math.hypot(vx, vy)
    if n == 0:
        return (0.0, 0.0)
    return (vx/n, vy/n)

def sub(a, b):
    return (a[0]-b[0], a[1]-b[1])

def add(a, b):
    return (a[0]+b[0], a[1]+b[1])

def mul(a, s):
    return (a[0]*s, a[1]*s)

def line_signed_distance_norm(p, a, b):
    """Signed distance from point p to infinite line through a->b, normalized by |b-a|.
       Positive sign is determined by 2D cross product orientation.
    """
    ax, ay = a
    bx, by = b
    px, py = p
    vx, vy = (bx-ax, by-ay)
    denom = math.hypot(vx, vy)
    if denom == 0:
        return 0.0
    # cross product (a->b) x (a->p) scaled by |v|
    return ((px-ax)*vy - (py-ay)*vx) / denom

def segment_projection_t(p, a, b):
    """Return scalar t for projection of p onto segment a->b: proj = a + t*(b-a)."""
    ax, ay = a
    bx, by = b
    px, py = p
    vx, vy = (bx-ax, by-ay)
    denom = (vx*vx + vy*vy)
    if denom == 0:
        return 0.0
    return ((px-ax)*vx + (py-ay)*vy) / denom

def point_in_quad(p, quad):
    """quad is list/tuple of 4 points. Use cv2.pointPolygonTest for robust check."""
    cnt = np.array(quad, dtype=np.int32)
    wn = cv2.pointPolygonTest(cnt, (float(p[0]), float(p[1])), False)
    return wn >= 0

def click_points(win, frame, num=4, title="Click 4 points: Start A, Start B, End A, End B"):
    pts = []
    clone = frame.copy()
    font = cv2.FONT_HERSHEY_SIMPLEX

    def draw_instructions(img, idx):
        text = [
            "Click in order:",
            "1) Start line endpoint A",
            "2) Start line endpoint B",
            "3) End line endpoint A",
            "4) End line endpoint B",
            f"Clicks: {idx}/4"
        ]
        y = 25
        for t in text:
            cv2.putText(img, t, (10, y), font, 0.6, (0, 0, 0), 3, cv2.LINE_AA)
            cv2.putText(img, t, (10, y), font, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
            y += 28

    def cb(event, x, y, flags, param):
        nonlocal pts, clone
        if event == cv2.EVENT_LBUTTONDOWN and len(pts) < num:
            pts.append((x, y))

    cv2.namedWindow(win, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(win, 1280, 720)
    cv2.setMouseCallback(win, cb)

    while True:
        disp = clone.copy()
        draw_instructions(disp, len(pts))
        # draw clicked points & partial lines
        for i, p in enumerate(pts):
            cv2.circle(disp, p, 6, (0, 255, 255), -1)
            cv2.putText(disp, f"P{i+1}", (p[0]+8, p[1]-8), font, 0.5, (0,0,0), 3, cv2.LINE_AA)
            cv2.putText(disp, f"P{i+1}", (p[0]+8, p[1]-8), font, 0.5, (0,255,255), 1, cv2.LINE_AA)
        if len(pts) >= 2:
            cv2.line(disp, pts[0], pts[1], (0, 200, 0), 2)  # start line
        if len(pts) >= 4:
            cv2.line(disp, pts[2], pts[3], (0, 0, 200), 2)  # end line
            # draw corridor polygon
            poly = np.array([pts[0], pts[1], pts[3], pts[2]], dtype=np.int32)
            cv2.polylines(disp, [poly], isClosed=True, color=(255, 140, 0), thickness=2)

        cv2.imshow(win, disp)
        key = cv2.waitKey(20) & 0xFF
        if key == 27:  # ESC to cancel
            pts = []
            break
        if len(pts) == num:
            break

    cv2.destroyWindow(win)
    if len(pts) != num:
        raise RuntimeError("Point selection cancelled.")
    return pts

def draw_overlay(frame, startA, startB, endA, endB, hip_pt, start_t, end_t, total_t, fps, show_axis=True):
    h, w = frame.shape[:2]
    font = cv2.FONT_HERSHEY_SIMPLEX
    # Lines
    cv2.line(frame, startA, startB, (0, 200, 0), 2)
    cv2.line(frame, endA, endB, (0, 0, 200), 2)
    # Corridor
    poly = np.array([startA, startB, endB, endA], dtype=np.int32)
    cv2.polylines(frame, [poly], isClosed=True, color=(255, 140, 0), thickness=2)
    # Points
    cv2.circle(frame, startA, 6, (0,255,0), -1)
    cv2.circle(frame, startB, 6, (0,255,0), -1)
    cv2.circle(frame, endA, 6, (0,0,255), -1)
    cv2.circle(frame, endB, 6, (0,0,255), -1)
    # Hip
    if hip_pt is not None:
        cv2.circle(frame, (int(hip_pt[0]), int(hip_pt[1])), 6, (0,255,255), -1)

    # Info text
    y0 = 28
    def put(s):
        nonlocal y0
        cv2.putText(frame, s, (10, y0), font, 0.7, (0,0,0), 3, cv2.LINE_AA)
        cv2.putText(frame, s, (10, y0), font, 0.7, (255,255,255), 1, cv2.LINE_AA)
        y0 += 30

    put("Green = Start line | Red = End line | Orange = ROI")
    if start_t is None:
        put("Start: --.-- s")
    else:
        put(f"Start: {start_t:.3f} s")
    if end_t is None:
        put("End:   --.-- s")
    else:
        put(f"End:   {end_t:.3f} s")
    if total_t is None:
        put("Δt:    --.-- s")
    else:
        put(f"Δt:    {total_t:.3f} s")

def main():
    import mediapipe as mp

    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open video: {VIDEO_PATH}")
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 0 or math.isnan(fps):
        fps = 30.0  # fallback
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Read first frame to collect clicks
    ret, first = cap.read()
    if not ret:
        raise RuntimeError("Could not read first frame.")
    clicks = click_points("Define Start/End", first, num=4,
                          title="Click 4 points: Start A, Start B, End A, End B")
    startA, startB, endA, endB = clicks[0], clicks[1], clicks[2], clicks[3]
    corridor = [startA, startB, endB, endA]

    # Lane axis for monotonicity (midpoints)
    m_start = ((startA[0]+startB[0]) * 0.5, (startA[1]+startB[1]) * 0.5)
    m_end   = ((endA[0]+endB[0]) * 0.5, (endA[1]+endB[1]) * 0.5)
    axis = unit(sub(m_end, m_start))
    # Bird's-eye homography (optional window)
    show_warp = False
    try:
        W = dist(startA, startB)
        # Approximate corridor "length" as distance between the two midpoints along axis
        L = dist(m_start, m_end)
        src = np.float32([startA, startB, endB, endA])  # clockwise from startA
        dst = np.float32([[0,0], [W,0], [W,L], [0,L]])
        H = cv2.getPerspectiveTransform(src, dst)
    except Exception:
        H = None

    # Prepare video writer if needed
    writer = None
    if WRITE_ANNOTATED_VIDEO:
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        writer = cv2.VideoWriter(OUT_VIDEO, fourcc, fps, (width, height))
        if not writer.isOpened():
            writer = None
            print("Warning: Could not open video writer; continuing without saving.")

    # MediaPipe Pose (single-person)
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(
        static_image_mode=False,
        model_complexity=MODEL_COMPLEXITY,
        enable_segmentation=False,
        min_detection_confidence=POSE_MIN_DET,
        min_tracking_confidence=POSE_MIN_TRK
    )

    # State vars
    hip_ema = None
    prev_sdist_start = None
    prev_sdist_end = None
    start_time_s = None
    end_time_s = None
    delta_time_s = None
    prev_progress = None  # dot((hip - m_start), axis)

    frame_idx = 0
    cv2.namedWindow("Measure", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Measure", 1280, 720)
    if H is not None:
        cv2.namedWindow("BirdsEye", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("BirdsEye", 640, 480)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        res = pose.process(rgb)

        hip_pt = None
        if res.pose_landmarks:
            lm = res.pose_landmarks.landmark
            # Use average of left/right hips if visible, else whichever confident
            # Landmarks 23: left_hip, 24: right_hip
            hps = []
            for idx in (23, 24):
                li = lm[idx]
                if 0.0 < li.x < 1.0 and 0.0 < li.y < 1.0 and li.visibility > 0.2:
                    hps.append((li.x * width, li.y * height))
            if len(hps) == 2:
                hip_pt = ((hps[0][0] + hps[1][0]) * 0.5, (hps[0][1] + hps[1][1]) * 0.5)
            elif len(hps) == 1:
                hip_pt = hps[0]

        # Smooth hip
        if hip_pt is not None:
            if hip_ema is None:
                hip_ema = hip_pt
            else:
                hip_ema = (hip_ema[0] * (1-EMA_ALPHA) + hip_pt[0] * EMA_ALPHA,
                           hip_ema[1] * (1-EMA_ALPHA) + hip_pt[1] * EMA_ALPHA)
        # Only consider detection when hip is inside corridor ROI
        consider = (hip_ema is not None) and point_in_quad(hip_ema, corridor)

        # Crossing logic
        if consider:
            # Signed distances to start/end infinite lines
            sdist_start = line_signed_distance_norm(hip_ema, startA, startB)
            sdist_end   = line_signed_distance_norm(hip_ema, endA, endB)
            # Projection params along segments
            t_start = segment_projection_t(hip_ema, startA, startB)
            t_end   = segment_projection_t(hip_ema, endA, endB)
            # Monotonic progress along lane axis
            prog = np.dot(np.array(hip_ema) - np.array(m_start), np.array(axis))

            # Detect start crossing if not yet set
            if start_time_s is None:
                if prev_sdist_start is not None:
                    if (prev_sdist_start > CROSSING_THRESH_PX and sdist_start < -CROSSING_THRESH_PX and
                        -SEGMENT_T_TOL <= t_start <= 1.0 + SEGMENT_T_TOL):
                        # forward direction check (hip moving from before start to after start)
                        # We expect progress to increase
                        if prev_progress is None or prog >= (prev_progress - 1.0):
                            start_time_s = frame_idx / fps
                            print(f"[INFO] Start crossed at {start_time_s:.3f} s (frame {frame_idx})")
                prev_sdist_start = sdist_start

            # Detect end crossing after start
            if start_time_s is not None and end_time_s is None:
                if prev_sdist_end is not None:
                    if (prev_sdist_end > CROSSING_THRESH_PX and sdist_end < -CROSSING_THRESH_PX and
                        -SEGMENT_T_TOL <= t_end <= 1.0 + SEGMENT_T_TOL):
                        # Ensure we've progressed sufficiently from start
                        if prog > dist(m_start, m_end) * 0.5:
                            end_time_s = frame_idx / fps
                            delta_time_s = end_time_s - start_time_s
                            print(f"[INFO] End crossed at {end_time_s:.3f} s (frame {frame_idx})")
                            print(f"[RESULT] Time between lines: {delta_time_s:.3f} s")
                prev_sdist_end = sdist_end

            prev_progress = prog

        # Draw overlay
        draw_overlay(frame, startA, startB, endA, endB, hip_ema, start_time_s, end_time_s, delta_time_s, fps)

        # Optional bird's-eye view
        if H is not None and show_warp:
            warped = cv2.warpPerspective(frame, H, (int(max(320, dist(startA, startB))), int(max(320, dist(m_start, m_end)))))
            cv2.imshow("BirdsEye", warped)

        cv2.imshow("Measure", frame)
        if writer is not None:
            writer.write(frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('r'):
            start_time_s = None
            end_time_s = None
            delta_time_s = None
            prev_sdist_start = None
            prev_sdist_end = None
            prev_progress = None
            print("[INFO] Reset start/end detections.")
        elif key == ord('w'):
            show_warp = not show_warp
            print(f"[INFO] Bird's-eye preview {'ON' if show_warp else 'OFF'}.")

    cap.release()
    if writer is not None:
        writer.release()
        print(f"[INFO] Annotated video saved: {OUT_VIDEO}")
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [31]:
"""
Sprinter time between two user-defined lines (start/end) using *toe* tracking.
- Click 4 points on the first displayed frame:
    1) Start line endpoint A
    2) Start line endpoint B
    3) End line endpoint A
    4) End line endpoint B
  The corridor (ROI) is the quad [startA, startB, endB, endA].
- Tracks the runner's toe (big toe tip = MediaPipe foot_index) rather than hip for sharper line crossing.
- Chooses the forward (leading) toe each frame based on progress along lane axis.
- Detects exact line crossings using sign-change + within-segment checks with linear interpolation
  to estimate sub-frame crossing times.
- Ignores detections outside the clicked corridor to avoid background runners.
- Optional bird's-eye (homography) window for visual confidence.
- Annotated video option.

Controls:
  q : quit
  r : reset start/end detections
  w : toggle bird's-eye view
"""
import cv2
import numpy as np
import math

# -------------- USER SETTINGS --------------
VIDEO_PATH = r"E:\Evoq\Sprinting video\IMG_0751.MOV"  # Change to your file path
WRITE_ANNOTATED_VIDEO = False            # Set True to save annotated MP4
OUT_VIDEO = "time_between_lines_TOE_annotated.mp4"

# MediaPipe Pose settings
POSE_MIN_DET = 0.5
POSE_MIN_TRK = 0.5
MODEL_COMPLEXITY = 1

# Smoothing for toe (exponential moving average)
EMA_ALPHA = 0.3

# Projection tolerance to accept a crossing as "within the segment" (t in [-tol, 1+tol])
SEGMENT_T_TOL = 0.02

# Require minimum landmark visibility
LM_VIS_THRESH = 0.5

# -------------------------------------------

def click_points(win, frame, num=4):
    pts = []
    clone = frame.copy()
    font = cv2.FONT_HERSHEY_SIMPLEX
    instructions = [
        "Click in order:",
        "1) Start line endpoint A",
        "2) Start line endpoint B",
        "3) End line endpoint A",
        "4) End line endpoint B",
        "(ESC to cancel)"
    ]
    def cb(event, x, y, flags, param):
        nonlocal pts
        if event == cv2.EVENT_LBUTTONDOWN and len(pts) < num:
            pts.append((x, y))
    cv2.namedWindow(win, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(win, 1280, 720)
    cv2.setMouseCallback(win, cb)
    while True:
        disp = clone.copy()
        y = 24
        for t in instructions:
            cv2.putText(disp, t, (10, y), font, 0.7, (0,0,0), 3, cv2.LINE_AA)
            cv2.putText(disp, t, (10, y), font, 0.7, (255,255,255), 1, cv2.LINE_AA)
            y += 30
        for i, p in enumerate(pts):
            cv2.circle(disp, p, 6, (0,255,255), -1)
            cv2.putText(disp, f"P{i+1}", (p[0]+8, p[1]-8), font, 0.6, (0,0,0), 3, cv2.LINE_AA)
            cv2.putText(disp, f"P{i+1}", (p[0]+8, p[1]-8), font, 0.6, (0,255,255), 1, cv2.LINE_AA)
        if len(pts) >= 2:
            cv2.line(disp, pts[0], pts[1], (0,200,0), 2)
        if len(pts) >= 4:
            cv2.line(disp, pts[2], pts[3], (0,0,200), 2)
            poly = np.array([pts[0], pts[1], pts[3], pts[2]], dtype=np.int32)
            cv2.polylines(disp, [poly], True, (255,140,0), 2)
        cv2.imshow(win, disp)
        k = cv2.waitKey(20) & 0xFF
        if k == 27:
            pts = []
            break
        if len(pts) == num:
            break
    cv2.destroyWindow(win)
    if len(pts) != num:
        raise RuntimeError("Point selection cancelled.")
    return pts

def unit(v):
    n = np.linalg.norm(v)
    return v / n if n > 0 else v

def line_signed_distance(p, a, b):
    # signed distance (pixels) to infinite line
    v = b - a
    n = np.linalg.norm(v)
    if n == 0:
        return 0.0
    return np.cross(v, p - a) / n

def segment_projection_t(p, a, b):
    v = b - a
    vv = (v*v).sum()
    if vv == 0:
        return 0.0
    return np.dot(p - a, v) / vv

def point_in_quad(p, quad):
    cnt = quad.astype(np.int32)
    return cv2.pointPolygonTest(cnt, (float(p[0]), float(p[1])), False) >= 0

def draw_overlay(frame, startA, startB, endA, endB, pt, start_t, end_t, delta_t):
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.line(frame, tuple(startA.astype(int)), tuple(startB.astype(int)), (0,200,0), 2)
    cv2.line(frame, tuple(endA.astype(int)),   tuple(endB.astype(int)),   (0,0,200), 2)
    poly = np.array([startA, startB, endB, endA], dtype=np.int32)
    cv2.polylines(frame, [poly], True, (255,140,0), 2)
    if pt is not None:
        cv2.circle(frame, (int(pt[0]), int(pt[1])), 6, (0,255,255), -1)
    y = 28
    def put(s):
        nonlocal y
        cv2.putText(frame, s, (10, y), font, 0.7, (0,0,0), 3, cv2.LINE_AA)
        cv2.putText(frame, s, (10, y), font, 0.7, (255,255,255), 1, cv2.LINE_AA)
        y += 30
    put("Green = Start, Red = End, Orange = ROI, Yellow = toe")
    put(f"Start: {start_t:.3f} s" if start_t is not None else "Start: --.-- s")
    put(f"End:   {end_t:.3f} s" if end_t is not None else "End:   --.-- s")
    put(f"Δt:    {delta_t:.3f} s" if delta_t is not None else "Δt:    --.-- s")

def main():
    import mediapipe as mp
    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open video: {VIDEO_PATH}")
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 0 or math.isnan(fps):
        fps = 30.0

    ok, first = cap.read()
    if not ok:
        raise RuntimeError("Cannot read first frame.")
    h, w = first.shape[:2]

    # Click lines
    p = click_points("Define Start/End", first, 4)
    startA, startB, endA, endB = map(lambda t: np.array(t, dtype=np.float32), p)
    corridor = np.array([startA, startB, endB, endA], dtype=np.float32)

    # Axis for forward progress (midpoint->midpoint)
    m_start = 0.5*(startA + startB)
    m_end   = 0.5*(endA + endB)
    axis = unit(m_end - m_start)

    # Homography for bird's-eye
    show_warp = False
    H = None
    try:
        W = np.linalg.norm(startB - startA)
        L = np.linalg.norm(m_end - m_start)
        src = np.float32([startA, startB, endB, endA])
        dst = np.float32([[0,0],[W,0],[W,L],[0,L]])
        H = cv2.getPerspectiveTransform(src, dst)
    except Exception:
        H = None

    writer = None
    if WRITE_ANNOTATED_VIDEO:
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        writer = cv2.VideoWriter(OUT_VIDEO, fourcc, fps, (w, h))
        if not writer.isOpened():
            writer = None
            print("[WARN] Could not open writer.")

    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(
        static_image_mode=False,
        model_complexity=MODEL_COMPLEXITY,
        enable_segmentation=False,
        min_detection_confidence=POSE_MIN_DET,
        min_tracking_confidence=POSE_MIN_TRK
    )

    # State
    toe_ema = None
    prev_toe = None
    prev_sdist_start = None
    prev_sdist_end = None
    start_time = None
    end_time = None
    delta_t = None
    prev_prog = None

    cv2.namedWindow("Measure", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Measure", 1280, 720)
    if H is not None:
        cv2.namedWindow("BirdsEye", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("BirdsEye", 640, 480)

    frame_idx = 0
    while True:
        ok, frame = cap.read()
        if not ok:
            break
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        res = pose.process(rgb)

        # --- select toe (leading foot) ---
        toe_pt = None
        if res.pose_landmarks:
            lm = res.pose_landmarks.landmark
            # MediaPipe indices: 31 left_foot_index, 32 right_foot_index
            cand = []
            for idx in (31, 32):
                li = lm[idx]
                if 0.0 < li.x < 1.0 and 0.0 < li.y < 1.0 and li.visibility >= LM_VIS_THRESH:
                    cand.append(np.array([li.x*w, li.y*h], dtype=np.float32))
            if cand:
                # choose the point with max progress along axis
                progs = [np.dot(c - m_start, axis) for c in cand]
                toe_pt = cand[int(np.argmax(progs))]

        # Smooth toe
        if toe_pt is not None:
            if toe_ema is None:
                toe_ema = toe_pt.copy()
            else:
                toe_ema = (1-EMA_ALPHA)*toe_ema + EMA_ALPHA*toe_pt

        # Only use if inside corridor
        consider = (toe_ema is not None) and point_in_quad(toe_ema, corridor)

        # Crossing detection with interpolation
        if consider:
            aS, bS = startA, startB
            aE, bE = endA, endB

            sdist_start = line_signed_distance(toe_ema, aS, bS)
            sdist_end   = line_signed_distance(toe_ema, aE, bE)

            tS = segment_projection_t(toe_ema, aS, bS)
            tE = segment_projection_t(toe_ema, aE, bE)

            prog = float(np.dot(toe_ema - m_start, axis))

            if prev_toe is not None:
                # interpolate sub-frame crossing time for start
                if start_time is None and prev_sdist_start is not None:
                    if tS >= -SEGMENT_T_TOL and tS <= 1.0 + SEGMENT_T_TOL:
                        if (prev_sdist_start > 0 and sdist_start < 0) or (prev_sdist_start < 0 and sdist_start > 0):
                            # linear interpolation factor between prev and current frame
                            denom = (sdist_start - prev_sdist_start)
                            if abs(denom) > 1e-6:
                                alpha = (0 - prev_sdist_start) / denom
                                # ensure monotonic forward movement
                                if prev_prog is None or prog >= prev_prog:
                                    start_time = (frame_idx - 1 + alpha) / fps
                                    print(f"[INFO] Start crossed at {start_time:.4f}s (frame ~{frame_idx-1+alpha:.2f})")

                # interpolate sub-frame crossing time for end
                if start_time is not None and end_time is None and prev_sdist_end is not None:
                    if tE >= -SEGMENT_T_TOL and tE <= 1.0 + SEGMENT_T_TOL:
                        if (prev_sdist_end > 0 and sdist_end < 0) or (prev_sdist_end < 0 and sdist_end > 0):
                            denom = (sdist_end - prev_sdist_end)
                            if abs(denom) > 1e-6:
                                alpha = (0 - prev_sdist_end) / denom
                                if prog > np.linalg.norm(m_end - m_start) * 0.5:
                                    end_time = (frame_idx - 1 + alpha) / fps
                                    delta_t = end_time - start_time
                                    print(f"[INFO] End crossed at {end_time:.4f}s (frame ~{frame_idx-1+alpha:.2f})")
                                    print(f"[RESULT] Time between lines: {delta_t:.4f}s")

            prev_sdist_start = sdist_start
            prev_sdist_end = sdist_end
            prev_prog = prog
            prev_toe = toe_ema.copy()
        else:
            prev_toe = None
            prev_sdist_start = None if start_time is None else prev_sdist_start
            prev_sdist_end = None if end_time is None else prev_sdist_end

        # Draw overlay
        draw_overlay(frame, startA, startB, endA, endB, toe_ema, start_time, end_time, delta_t)

        # Bird's-eye view
        if H is not None and show_warp:
            Wv = max(int(np.linalg.norm(startB - startA)), 320)
            Lv = max(int(np.linalg.norm(m_end - m_start)), 320)
            warped = cv2.warpPerspective(frame, H, (Wv, Lv))
            cv2.imshow("BirdsEye", warped)

        cv2.imshow("Measure", frame)
        if writer is not None:
            writer.write(frame)

        k = cv2.waitKey(1) & 0xFF
        if k == ord('q'):
            break
        elif k == ord('r'):
            start_time = None
            end_time = None
            delta_t = None
            prev_toe = None
            prev_sdist_start = None
            prev_sdist_end = None
            prev_prog = None
            print("[INFO] Reset detections.")
        elif k == ord('w'):
            show_warp = not show_warp
            print(f"[INFO] Bird's-eye {'ON' if show_warp else 'OFF'}.")

    cap.release()
    if writer is not None:
        writer.release()
        print(f"[INFO] Saved annotated video to: {OUT_VIDEO}")
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()